### Konstantina Andronikou

## Transitive Vs Intransitive Verbs 

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Intransitive 

In [4]:
# remove warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Carla pays")
output

{'verbs': [{'verb': 'pays',
   'description': '[ARG0: Carla] [V: pays]',
   'tags': ['B-ARG0', 'B-V']}],
 'words': ['Carla', 'pays']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Carla pays"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'pays',
     'description': '[ARG0: Carla] [V: pays]',
     'tags': ['B-ARG0', 'B-V']}],
   'words': ['Carla', 'pays']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays', meta=True, nsamples=1000, remove_duplicates = True)

for k, v in t.items():
    print(k,v)

meta [{'first_name': 'Cynthia'}, {'first_name': 'Karen'}, {'first_name': 'Frank'}, {'first_name': 'Jack'}, {'first_name': 'Jack'}, {'first_name': 'Elaine'}, {'first_name': 'John'}, {'first_name': 'Chris'}, {'first_name': 'Marilyn'}, {'first_name': 'Jill'}, {'first_name': 'Joan'}, {'first_name': 'Alison'}, {'first_name': 'Claire'}, {'first_name': 'Emma'}, {'first_name': 'Nicole'}, {'first_name': 'Victoria'}, {'first_name': 'Albert'}, {'first_name': 'Tony'}, {'first_name': 'Edwin'}, {'first_name': 'Bruce'}, {'first_name': 'Carolyn'}, {'first_name': 'Lauren'}, {'first_name': 'Larry'}, {'first_name': 'Jack'}, {'first_name': 'Julie'}, {'first_name': 'Simon'}, {'first_name': 'Diane'}, {'first_name': 'Grace'}, {'first_name': 'Adam'}, {'first_name': 'Julia'}, {'first_name': 'Sophie'}, {'first_name': 'Greg'}, {'first_name': 'Edward'}, {'first_name': 'James'}, {'first_name': 'Emma'}, {'first_name': 'Suzanne'}, {'first_name': 'Karen'}, {'first_name': 'Steven'}, {'first_name': 'Alan'}, {'first_nam

In [12]:
with open('Dataset_BiLSTM/Intransitive.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
test = MFT(**t, name = 'detect_arg0name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Intransitive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    12 (1.2%)

Example fails:
[ARG1: Alexandra] [V: pays]
----
[ARG1: Alexandra] [V: pays]
----
[ARG1: Alexandra] [V: pays]
----
['Cynthia', 'pays'] [ True]
['Karen', 'pays'] [ True]
['Frank', 'pays'] [ True]
['Jack', 'pays'] [ True]
['Jack', 'pays'] [ True]
['Elaine', 'pays'] [ True]
['John', 'pays'] [ True]
['Chris', 'pays'] [ True]
['Marilyn', 'pays'] [ True]
['Jill', 'pays'] [ True]
['Joan', 'pays'] [ True]
['Alison', 'pays'] [ True]
['Claire', 'pays'] [ True]
['Emma', 'pays'] [ True]
['Nicole', 'pays'] [ True]
['Victoria', 'pays'] [ True]
['Albert', 'pays'] [ True]
['Tony', 'pays'] [ True]
['Edwin', 'pays'] [ True]
['Bruce', 'pays'] [ True]
['Carolyn', 'pays'] [ True]
['Lauren', 'pays'] [ True]
['Larry', 'pays'] [ True]
['Jack', 'pays'] [ True]
['Julie', 'pays'] [ True]
['Simon', 'pays'] [ True]
['Diane', 'pays'] [ True]
['Grace', 'pays'] [ True]
['Adam', 'pays'] [ True]
['Julia', 'pays'] [ True]
['Sophie', 'pays'] [ True

In [15]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Intransitive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive 

In [16]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [17]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [18]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [19]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [20]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Alexander'}, {'first_name': 'Katie'}, {'first_name': 'Richard'}, {'first_name': 'Heather'}, {'first_name': 'Tim'}, {'first_name': 'Sue'}, {'first_name': 'Jeff'}, {'first_name': 'David'}, {'first_name': 'Joseph'}, {'first_name': 'Alexandra'}, {'first_name': 'Lisa'}, {'first_name': 'Claire'}, {'first_name': 'Robin'}, {'first_name': 'Virginia'}, {'first_name': 'Ed'}, {'first_name': 'Donald'}, {'first_name': 'Brian'}, {'first_name': 'Patricia'}, {'first_name': 'Harriet'}, {'first_name': 'Nicole'}, {'first_name': 'Matt'}, {'first_name': 'Paul'}, {'first_name': 'Lisa'}, {'first_name': 'Anthony'}, {'first_name': 'Katherine'}, {'first_name': 'Chris'}, {'first_name': 'Hugh'}, {'first_name': 'Jack'}, {'first_name': 'Ray'}, {'first_name': 'Jack'}, {'first_name': 'Jennifer'}, {'first_name': 'Jennifer'}, {'first_name': 'Charles'}, {'first_name': 'Edward'}, {'first_name': 'Sally'}, {'first_name': 'Julia'}, {'first_name': 'Heather'}, {'first_name': 'Diane'}, {'first_name': 'Sara

In [21]:
with open('Dataset_BiLSTM/Transitive.txt', 'w') as f:
    print(t.data, file = f)

In [22]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Transitive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    25 (2.5%)

Example fails:
[ARG1: Lawrence] [V: pays] [ARG3: for coffee] .
----
[ARG1: Margaret] [V: pays] [ARG3: for coffee] .
----
[ARG1: Alison] [V: pays] [ARG3: for coffee] .
----
['Alexander', 'pays', 'for', 'coffee', '.'] [ True]
['Katie', 'pays', 'for', 'coffee', '.'] [ True]
['Richard', 'pays', 'for', 'coffee', '.'] [ True]
['Heather', 'pays', 'for', 'coffee', '.'] [ True]
['Tim', 'pays', 'for', 'coffee', '.'] [ True]
['Sue', 'pays', 'for', 'coffee', '.'] [ True]
['Jeff', 'pays', 'for', 'coffee', '.'] [ True]
['David', 'pays', 'for', 'coffee', '.'] [ True]
['Joseph', 'pays', 'for', 'coffee', '.'] [ True]
['Alexandra', 'pays', 'for', 'coffee', '.'] [False]
['Lisa', 'pays', 'for', 'coffee', '.'] [ True]
['Claire', 'pays', 'for', 'coffee', '.'] [ True]
['Robin', 'pays', 'for', 'coffee', '.'] [ True]
['Virginia', 'pays', 'for', 'coffee', '.'] [ True]
['Ed', 'pays', 'for', 'coffee', '.'] [ True]
['Donald', 'pays', 'for',

In [23]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Transitive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive-Intransitive Robustness Test

In [24]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Martha'}, {'first_name': 'Katie'}, {'first_name': 'Fiona'}, {'first_name': 'Pamela'}, {'first_name': 'Lawrence'}, {'first_name': 'Robert'}, {'first_name': 'Eleanor'}, {'first_name': 'Christine'}, {'first_name': 'Karen'}, {'first_name': 'Alexandra'}, {'first_name': 'Deborah'}, {'first_name': 'Nancy'}, {'first_name': 'Patrick'}, {'first_name': 'Frederick'}, {'first_name': 'Harry'}, {'first_name': 'Christopher'}, {'first_name': 'Alison'}, {'first_name': 'Betty'}, {'first_name': 'Sara'}, {'first_name': 'Martin'}, {'first_name': 'Harry'}, {'first_name': 'Johnny'}, {'first_name': 'Harriet'}, {'first_name': 'Hugh'}, {'first_name': 'Gary'}, {'first_name': 'Anthony'}, {'first_name': 'Walter'}, {'first_name': 'Anne'}, {'first_name': 'Donald'}, {'first_name': 'Fred'}, {'first_name': 'Carol'}, {'first_name': 'Marie'}, {'first_name': 'Billy'}, {'first_name': 'Julie'}, {'first_name': 'Andrea'}, {'first_name': 'Catherine'}, {'first_name': 'Alexander'}, {'first_name': 'Joan'}, {'

In [25]:
with open('Dataset_BiLSTM/Intransitive_Rob.txt', 'w') as f:
    print(t.data, file = f)

In [26]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Intransitive_Rob.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    349 (34.9%)

Example fails:
[ARGM-DIS: Steve] [V: payed] .
----
Louise [V: payed] .
----
Rebecca [V: payed] .
----
['Martha', 'payed', '.'] [False]
['Katie', 'payed', '.'] [ True]
['Fiona', 'payed', '.'] [ True]
['Pamela', 'payed', '.'] [ True]
['Lawrence', 'payed', '.'] [ True]
['Robert', 'payed', '.'] [ True]
['Eleanor', 'payed', '.'] [ True]
['Christine', 'payed', '.'] [ True]
['Karen', 'payed', '.'] [ True]
['Alexandra', 'payed', '.'] [False]
['Deborah', 'payed', '.'] [ True]
['Nancy', 'payed', '.'] [False]
['Patrick', 'payed', '.'] [False]
['Frederick', 'payed', '.'] [ True]
['Harry', 'payed', '.'] [False]
['Christopher', 'payed', '.'] [False]
['Alison', 'payed', '.'] [False]
['Betty', 'payed', '.'] [ True]
['Sara', 'payed', '.'] [False]
['Martin', 'payed', '.'] [ True]
['Harry', 'payed', '.'] [False]
['Johnny', 'payed', '.'] [ True]
['Harriet', 'payed', '.'] [ True]
['Hugh', 'payed', '.'] [ True]
['Gary', 'payed', '.

In [27]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Intransitive_Rob.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive

In [28]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Dick'}, {'first_name': 'Catherine'}, {'first_name': 'Gary'}, {'first_name': 'Steve'}, {'first_name': 'Judy'}, {'first_name': 'Julie'}, {'first_name': 'Kathleen'}, {'first_name': 'Ian'}, {'first_name': 'Harold'}, {'first_name': 'Charlotte'}, {'first_name': 'Michael'}, {'first_name': 'Eleanor'}, {'first_name': 'Alison'}, {'first_name': 'Rebecca'}, {'first_name': 'Alice'}, {'first_name': 'Stephen'}, {'first_name': 'Frank'}, {'first_name': 'Scott'}, {'first_name': 'Joan'}, {'first_name': 'Rebecca'}, {'first_name': 'Diana'}, {'first_name': 'Joseph'}, {'first_name': 'Tony'}, {'first_name': 'Don'}, {'first_name': 'Dan'}, {'first_name': 'Kevin'}, {'first_name': 'Angela'}, {'first_name': 'Peter'}, {'first_name': 'Ed'}, {'first_name': 'Kathryn'}, {'first_name': 'Claire'}, {'first_name': 'Claire'}, {'first_name': 'Linda'}, {'first_name': 'Frederick'}, {'first_name': 'Jessica'}, {'first_name': 'Jay'}, {'first_name': 'Louis'}, {'first_name': 'Ken'}, {'first_name': 'Fiona'}, {'

In [29]:
with open('Dataset_BiLSTM/Transitive_Rob.txt', 'w') as f:
    print(t.data, file = f)

In [30]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Transitive_Rob.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    514 (51.4%)

Example fails:
[ARGM-TMP: Richard] [V: payed] [ARG1: for coffee] .
----
[ARG2: Michelle] [V: payed] [ARG1: for coffee] .
----
[ARG2: Christine] [V: payed] [ARG1: for coffee] .
----
['Dick', 'payed', 'for', 'coffee', '.'] [ True]
['Catherine', 'payed', 'for', 'coffee', '.'] [ True]
['Gary', 'payed', 'for', 'coffee', '.'] [ True]
['Steve', 'payed', 'for', 'coffee', '.'] [False]
['Judy', 'payed', 'for', 'coffee', '.'] [False]
['Julie', 'payed', 'for', 'coffee', '.'] [False]
['Kathleen', 'payed', 'for', 'coffee', '.'] [False]
['Ian', 'payed', 'for', 'coffee', '.'] [False]
['Harold', 'payed', 'for', 'coffee', '.'] [ True]
['Charlotte', 'payed', 'for', 'coffee', '.'] [ True]
['Michael', 'payed', 'for', 'coffee', '.'] [False]
['Eleanor', 'payed', 'for', 'coffee', '.'] [ True]
['Alison', 'payed', 'for', 'coffee', '.'] [False]
['Rebecca', 'payed', 'for', 'coffee', '.'] [ True]
['Alice', 'payed', 'for', 'coffee', '.'] [

In [33]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Transitive_Rob.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 